<a href="https://colab.research.google.com/github/vijji5432/Timetable_generator/blob/main/Vizuara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd
import random
import xlsxwriter

# Step 1: Load the Excel file and read the sheets
data_path = '/content/TimeTable Project.xlsx'
data = pd.ExcelFile(data_path)

In [ ]:
# Step 2: Read the sheets for period allotment and teacher-subject mapping
period_allotment = data.parse('Period Allotment')
teacher_subject_mapping = data.parse('Teacher-Subject Mapping')
home_teachers_df = data.parse('Teacher Subject Mapping 2')
teacher_subject_mapping.columns = teacher_subject_mapping.columns.str.strip()
teacher_subject_mapping.rename(columns={"I A": "IA"}, inplace=True)

In [ ]:
# Step 3: Inspect the sheets to ensure data is loaded correctly
period_allotment['Jr. KG'] = pd.to_numeric(period_allotment['Jr. KG'], errors='coerce')
period_allotment['Sr. KG'] = pd.to_numeric(period_allotment['Sr. KG'], errors='coerce')
period_allotment['I'] = pd.to_numeric(period_allotment['I'], errors='coerce')
period_allotment.fillna(0, inplace=True)
period_allotment = period_allotment[period_allotment['Subject'].apply(lambda x: isinstance(x, str) and x.strip() != "0")]


In [ ]:
# Get unique subject names from both DataFrames
period_allotment_subjects = set(period_allotment['Subject'].unique())
teacher_subject_mapping_subjects = set(teacher_subject_mapping['Subject'].unique())

# Find subjects in Period Allotment not in Teacher-Subject Mapping
subjects_not_in_mapping = period_allotment_subjects - teacher_subject_mapping_subjects

# Find subjects in Teacher-Subject Mapping not in Period Allotment
subjects_not_in_allotment = teacher_subject_mapping_subjects - period_allotment_subjects

# Display mismatches
if subjects_not_in_mapping or subjects_not_in_allotment:
    print("Subjects in Period Allotment not in Teacher-Subject Mapping:")
    print(subjects_not_in_mapping if subjects_not_in_mapping else "None")

    print("\nSubjects in Teacher-Subject Mapping not in Period Allotment:")
    print(subjects_not_in_allotment if subjects_not_in_allotment else "None")
else:
    print("All subject names are in sync between Period Allotment and Teacher-Subject Mapping.")


Subjects in Period Allotment not in Teacher-Subject Mapping:
{'Yoga', 'PLAY PEN ', 'Role play', 'Marathi ', 'PE/ CCA Sports', 'Mathematic ', 'Music T', 'CCA-CREATIVE', 'Environmental Education --EE', 'KARATE ', 'CLASS TESTS ', 'Life Skill', 'SGI'}

Subjects in Teacher-Subject Mapping not in Period Allotment:
{'Economics ', 'State Level Marathi ', nan, 'Applied Math ', 'SUPW', 'POL SCIENCE ', 'PHYSICS', 'Legal Studies ', 'Marathi -3rd Lang', 'GEOG ', 'Enterpreneurship', 'Mathematics', 'FM', 'HIST ', 'Sanskrit ', 'AI', 'Accountancy', 'Environmental Education', 'Mass Media ', 'Buisness Studies ', 'KARATE', 'YOGA', 'ESL ', 'BIO', 'Life skills ', 'PSYCHOLOGY ', 'SPANISH', 'CHEM', 'General studies ', 'Commercial  Art', 'SAP/ Career Counselling ', 'Physical EDUCATION', 'CIVICS', 'Computer Science'}


In [ ]:
# Step 4: Preprocess the data
period_allotment = period_allotment.fillna(0)

In [ ]:
# Step 5: Define key constraints
PERIODS_PER_DAY = 10
DAYS_PER_WEEK = 5
PERIOD_DURATION = 30

columns = ['1st', '2nd','3rd', '4th','5th','break', '6th', '7th','dispersal']
rows = ['mon', 'tue', 'wed', 'thu', 'fri']
columns1 = ['HomeRoom', '1st', '2nd','break', '3rd', '4th','5th', '6th', '7th','8th','break','9th','10th','dispersal']

In [ ]:
# Step 6: Create a blank timetable structure
def initialize_timetable(classes, sections):
    timetable = {}
    for cls in classes:
        timetable[cls] = {}
        for section in sections:
            if cls in ['Jr. KG', 'Sr. KG']:
                timetable[cls][section] = pd.DataFrame(' ', index=rows, columns=columns)
            elif cls == 'I':
                timetable[cls][section] = pd.DataFrame(' ', index=rows, columns=columns1)
    return timetable

def fill_common_columns(timetable):
    common_columns = {
        'break': 'Break',
        'dispersal': 'Dispersal',
        'HomeRoom': 'HomeRoom'
    }
    for cls, sections in timetable.items():
        for section, df in sections.items():
            for column, value in common_columns.items():
                if column in df.columns:
                    df[column] = value

    return timetable

# Define the fixed assembly schedule
def add_assembly(timetable):
    assembly_schedule = {
        'Jr. KG': {'day': 'tue', 'period': '2nd'},
        'Sr. KG': {'day': 'tue', 'period': '2nd'},
        'I': {'day': 'tue', 'period': '5th'}
    }
    for cls, sections in timetable.items():
        if cls in assembly_schedule:
            assembly_day = assembly_schedule[cls]['day']
            assembly_period = assembly_schedule[cls]['period']
            for section, df in sections.items():
                if assembly_day in df.index and assembly_period in df.columns:
                    df.loc[assembly_day, assembly_period] = 'Assembly'
    return timetable

In [ ]:
classes = ['Jr. KG', 'Sr. KG', 'I']
sections = ['A', 'B', 'C', 'D']
timetable = initialize_timetable(classes, sections)

In [ ]:
def allocate_subjects(timetable, period_allotment):
    period_allotment = {subject: freq for subject, freq in period_allotment.items() if subject != 'Assembly' and freq != 0.5}
    for subject, frequency in period_allotment.items():
        periods_allocated = 0
        for slot in timetable.columns:
            if slot in ['break', 'dispersal', 'HomeRoom']:
                continue
            for day in timetable.index:
                if periods_allocated >= frequency:
                    break
                if timetable.at[day, slot].strip() == '':
                    timetable.at[day, slot] = subject
                    periods_allocated += 1
                if periods_allocated >= frequency:
                    break
            if periods_allocated >= frequency:
                break
    return timetable

In [ ]:

def assign_teachers(timetable, teacher_subject_mapping):
    for cls in classes:
        for section in sections:
            class_section = f"{cls}{' ' if cls != 'I' else ''}{section}"
            if class_section not in teacher_subject_mapping.columns:
                print(f"Class-Section {class_section} not found in teacher_subject_mapping.")
                continue
            for day in rows:
                for col in columns:
                    if col not in ['break', 'dispersal', 'HomeRoom']:
                        subject = timetable[cls][section].loc[day, col]
                        if subject != '':
                            # print("in")
                            teachers = teacher_subject_mapping.loc[
                                teacher_subject_mapping['Subject'] == subject,
                                class_section
                            ].values
                            if teachers.size > 0:
                                timetable[cls][section].loc[day, col] += f" ({teachers[0]})"

In [ ]:
def pair_half_frequency_subjects(period_allotment):
    paired_subjects = {}
    for cls in ['Jr. KG', 'Sr. KG', 'I']:
        half_freq_subjects = period_allotment[['Subject', cls]]
        subjects = half_freq_subjects[half_freq_subjects[cls] == 0.5]['Subject'].dropna().tolist()
        paired_subjects[cls] = [
            (subjects[i], subjects[i+1])
            for i in range(0, len(subjects) - 1, 2)
        ]
        if len(subjects) % 2 == 1:
            paired_subjects[cls].append((subjects[-1], subjects[-1]))
    return paired_subjects
paired_subjects = pair_half_frequency_subjects(period_allotment)
print(paired_subjects)

def allocate_combined_half_frequency_subjects(timetable, paired_subjects):
    for cls, pairs in paired_subjects.items():
        combined_pairs = [f"{pair[0]} / {pair[1]}" for pair in pairs]
        print(f"Processing class: {cls}")
        print(f"Combined pairs for {cls}: {combined_pairs}")
        for section, days in timetable[cls].items():
            print(f"Processing {cls} - {section}: {days}")
            allocated_pairs = set()
            for day, periods in days.items():
                if day in ['break', 'dispersal']:
                    print(f"Skipping {day} for {cls} as it is not a class day.")
                    continue
                print(f"Processing day: {day}")
                empty_slots = [i for i, period in enumerate(periods) if period.strip() == '']
                print(f"Empty slots for {cls} on {day}: {empty_slots}")
                if not empty_slots:
                    print(f"No empty slots for {cls} on {day}, skipping.")
                    continue
                for pair in combined_pairs:
                    if pair not in allocated_pairs and empty_slots:
                        periods[empty_slots[0]] = pair
                        print(f"Allocated '{pair}' to {cls} on {day}, slot {empty_slots[0]}")
                        allocated_pairs.add(pair)
                        empty_slots.pop(0)
                        if len(allocated_pairs) == len(combined_pairs):
                            print(f"All pairs allocated for {cls} on {day}")
                            break
                for i in empty_slots:
                    periods[i] = "leisure"

    return timetable

{'Jr. KG': [('Computer Studies', 'Circle Time'), ('Life Skill', 'Yoga'), ('PLAY PEN ', 'Role play')], 'Sr. KG': [('Computer Studies', 'Circle Time'), ('Life Skill', 'Yoga'), ('DEED', 'Assembly'), ('PLAY PEN ', 'PLAY PEN ')], 'I': []}


In [ ]:
add_assembly(timetable)
for cls in classes:
    for section in sections:
        if cls in period_allotment.columns:
            period_allotment_class = period_allotment[['Subject', cls]]
            period_allotment_class = period_allotment_class.set_index('Subject').to_dict()[cls]
            timetable[cls][section] = allocate_subjects(timetable[cls][section], ) period_allotment_class
        else:
            print(f"Class {cls} not found in period_allotment columns")
timetable = allocate_combined_half_frequency_subjects(timetable, paired_subjects)
assign_teachers(timetable, teacher_subject_mapping)
fill_common_columns(timetable)

Processing class: Jr. KG
Combined pairs for Jr. KG: ['Computer Studies / Circle Time', 'Life Skill / Yoga', 'PLAY PEN  / Role play']
Processing Jr. KG - A:      1st       2nd          3rd             4th             5th break  \
mon  Eng     Hindi  Mathematic              EVS  Clay modelling         
tue  Eng  Assembly  Mathematic   PE/ CCA Sports           Music         
wed  Eng     Hindi  Mathematic   PE/ CCA Sports           Dance         
thu  Eng     Hindi  Mathematic             Art          Library         
fri  Eng     Hindi          EVS            Art             DEED         

                              6th 7th dispersal  
mon                   Wonder time                
tue  Environmental Education --EE                
wed  Environmental Education --EE                
thu                                              
fri                                              
Processing day: 1st
Empty slots for Jr. KG on 1st: []
No empty slots for Jr. KG on 1st, skipping.
Process

<ipython-input-94-da2118e2a8c7>:36: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-94-da2118e2a8c7>:36: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-94-da2118e2a8c7>:36: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-94-da2118e2a8c7>:44: FutureWarning: Series.__setitem__ treating key

{'Jr. KG': {'A':                             1st             2nd          3rd             4th  \
  mon  Eng (Nupur FT , Pooja  R )      Hindi (HR)  Mathematic         EVS (HR)   
  tue  Eng (Nupur FT , Pooja  R )  Assembly (nan)  Mathematic   PE/ CCA Sports   
  wed  Eng (Nupur FT , Pooja  R )      Hindi (HR)  Mathematic   PE/ CCA Sports   
  thu  Eng (Nupur FT , Pooja  R )      Hindi (HR)  Mathematic        Art  (HR)   
  fri  Eng (Nupur FT , Pooja  R )      Hindi (HR)     EVS (HR)       Art  (HR)   
  
                             5th  break                             6th  \
  mon  Clay modelling (Shridevi)  Break         Wonder time (Neesha.R )   
  tue           Music (Sanjukta)  Break    Environmental Education --EE   
  wed            Dance (Pooja B)  Break    Environmental Education --EE   
  thu          Library (Pooja S)  Break  Computer Studies / Circle Time   
  fri                DEED (Isha)  Break               Life Skill / Yoga   
  
                         7th  dispers

In [ ]:
 # Create a mock DataFrame for home room teachers
def assign_teachers_with_alternate_hr(timetable, teacher_subject_mapping):
    data = {
    'Class-Section': ['Jr. KG A', 'Jr. KG B', 'Jr. KG C', 'Jr. KG D', 'Sr. KG A', 'Sr. KG B', 'Sr. KG C', 'Sr. KG D', 'I A', 'I B', 'I C', 'I D'],
    'Teachers': [['Kim', 'Rukhsaar'], ['Maria', 'Shital'], ['Shireen', 'Uphaar'], ['Priyanka', 'Darshana'],
                 ['Pooja', 'Nupur'], ['Anjum', 'Lakshmipriya'], ['Shubha', 'Jyoti'], ['Ankita', 'Bhakti'],
                 ['Remya', 'Saraswati'], ['Kanak', 'Matilda'], ['Rishita', 'Shashwati'], ['Charu', 'Maariyah']]
    }
    home_teachers_df = pd.DataFrame(data)
    home_teachers_mapping = dict(zip(home_teachers_df['Class-Section'], home_teachers_df['Teachers']))
    classes = list(timetable.keys())
    sections = list(timetable[classes[0]].keys())
    for cls in classes:
        for section in sections:
            class_section = f"{cls} {section}"
            if class_section not in home_teachers_mapping:
                print(f"Class-Section {class_section} not found in home_teachers_mapping.")
                continue
            hr_teachers = home_teachers_mapping[class_section]
            if len(hr_teachers) < 2:
                print(f"Not enough teachers for HR mapping in {class_section}.")
                continue
            alternate_teacher_index = 0
            rows = timetable[cls][section].index
            columns = timetable[cls][section].columns

            for day in rows:
                for col in columns:
                    if col not in ['break', 'dispersal', 'HomeRoom']:
                        subject = timetable[cls][section].loc[day, col]
                        if "HR" in subject:
                            teacher = hr_teachers[alternate_teacher_index]
                            timetable[cls][section].loc[day, col] = subject.replace("HR", teacher)
                            alternate_teacher_index = (alternate_teacher_index + 1) % len(hr_teachers)
    return timetable
assign_teachers_with_alternate_hr(timetable=timetable, teacher_subject_mapping=teacher_subject_mapping)

{'Jr. KG': {'A':                             1st               2nd         3rd             4th  \
  mon  eng (Nupur FT , Pooja  R )       hindi (Kim)  mathematic  evs (Rukhsaar)   
  tue  eng (Nupur FT , Pooja  R )          Assembly  mathematic  pe/ cca sports   
  wed  eng (Nupur FT , Pooja  R )       hindi (Kim)  mathematic  pe/ cca sports   
  thu  eng (Nupur FT , Pooja  R )  hindi (Rukhsaar)  mathematic       art (Kim)   
  fri  eng (Nupur FT , Pooja  R )  hindi (Rukhsaar)   evs (Kim)  art (Rukhsaar)   
  
                             5th  break                             6th  \
  mon  clay modelling (Shridevi)  Break                  assembly (nan)   
  tue           music (Sanjukta)  Break         wonder time (Neesha.R )   
  wed            dance (Pooja B)  Break    environmental education --ee   
  thu          library (Pooja S)  Break    environmental education --ee   
  fri                deed (Isha)  Break  computer studies / circle time   
  
                        7th  di

In [ ]:
# Step 9: Save the timetables to Excel
writer = pd.ExcelWriter('/content/Generated_Timetables.xlsx', engine='xlsxwriter')

for cls in classes:
    for section in sections:
        timetable[cls][section].to_excel(writer, sheet_name=f"{cls}_{section}")

writer.close()
print("Timetables generated and saved successfully.")

In [ ]:
def validate_period_frequencies(timetable, period_allotment):
    for cls in timetable:
        for section in timetable[cls]:
            # Iterate over all days and slots in the timetable
            for day in timetable[cls][section].index:
                for slot in timetable[cls][section].columns:
                    if slot in ['break', 'dispersal', 'HR']:  # Skip non-teaching periods
                        continue

                    subject = timetable[cls][section].at[day, slot]

                    if subject != '':  # Skip empty slots
                        # Get the expected frequency from period_allotment
                        frequency = period_allotment.loc[period_allotment['Subject'] == subject, cls].values

                        if frequency.size > 0:  # Check if frequency exists for the subject
                            expected_frequency = frequency[0]

                            # Count the actual frequency of the subject in the timetable
                            actual_frequency = (timetable[cls][section] == subject).sum().sum()

                            # Compare actual and expected frequencies
                            if actual_frequency != expected_frequency:
                                print(f"Warning: Frequency mismatch for {subject} in {cls}-{section}.\n"
                                      f"Expected: {expected_frequency}, Found: {actual_frequency}")
                        else:
                            print(f"Warning: {subject} is not found in period_allotment for {cls}-{section}.")
                    else:
                        continue  # Skip empty slots



validate_period_frequencies(timetable, period_allotment)


Expected: 4.5, Found: 5
Expected: 0.5, Found: 1
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 0.5, Found: 1
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 0.5, Found: 1
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 0.5, Found: 1
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5
Expected: 4.5, Found: 5


In [ ]:
import pandas as pd
import random
import xlsxwriter

# Step 1: Load the Excel file and read the sheets
data_path = '/content/TimeTable Project.xlsx'
data = pd.ExcelFile(data_path)

# Step 2: Read the sheets for period allotment and teacher-subject mapping
period_allotment = data.parse('Period Allotment')  # Adjust the sheet name as per your file
teacher_subject_mapping = data.parse('Teacher-Subject Mapping')  # Adjust the sheet name as per your file
home_teachers_df = data.parse('Teacher Subject Mapping 2')
teacher_subject_mapping.columns = teacher_subject_mapping.columns.str.strip()
teacher_subject_mapping.rename(columns={"I A": "IA"}, inplace=True)


# Step 3: Inspect the sheets to ensure data is loaded correctly
period_allotment['Jr. KG'] = pd.to_numeric(period_allotment['Jr. KG'], errors='coerce')
period_allotment['Sr. KG'] = pd.to_numeric(period_allotment['Sr. KG'], errors='coerce')
period_allotment['I'] = pd.to_numeric(period_allotment['I'], errors='coerce')
period_allotment.fillna(0, inplace=True)
period_allotment = period_allotment[period_allotment['Subject'].apply(lambda x: isinstance(x, str) and x.strip() != "0")]

#print(period_allotment.isna().sum())
#print("Period Allotment\n", period_allotment.head())
#print("Teacher Subject Mapping\n", teacher_subject_mapping.head())
# Clean up the teacher names: strip extra spaces from each teacher's name using .apply()


# Step 4: Preprocess the data
# Convert period allotment into a usable format
period_allotment = period_allotment.fillna(0)



# Step 5: Define key constraints
# Number of periods per day and other constraints
PERIODS_PER_DAY = 10  # Excluding breaks and dispersal
DAYS_PER_WEEK = 5
PERIOD_DURATION = 30  # in minutes

#empty table structure
columns = ['1st', '2nd','3rd', '4th','5th','break', '6th', '7th','dispersal']
rows = ['mon', 'tue', 'wed', 'thu', 'fri']
columns1 = ['HomeRoom', '1st', '2nd','break', '3rd', '4th','5th', '6th', '7th','8th','break','9th','10th','dispersal']

# Step 6: Create a blank timetable structure
def initialize_timetable(classes, sections):
    timetable = {}
    for cls in classes:
        timetable[cls] = {}
        for section in sections:
            if cls in ['Jr. KG', 'Sr. KG']:
                # Jr.KG and Sr.KG timetable structure
                timetable[cls][section] = pd.DataFrame(' ', index=rows, columns=columns)
            elif cls == 'I':
                # Class I timetable structure
                timetable[cls][section] = pd.DataFrame(' ', index=rows, columns=columns1)
    return timetable

def fill_common_columns(timetable):
    # Define the common columns and their respective values
    common_columns = {
        'break': 'Break',
        'dispersal': 'Dispersal',
        'HomeRoom': 'HomeRoom'
    }

    # Iterate over all classes in the timetable
    for cls, sections in timetable.items():
        # Iterate over all sections in each class
        for section, df in sections.items():
            # Update common columns for the current DataFrame
            for column, value in common_columns.items():
                if column in df.columns:  # Check if the column exists in the DataFrame
                    df[column] = value  # Fill the entire column with the respective value

    return timetable

def add_assembly(timetable):
    # Define the fixed assembly schedule
    assembly_schedule = {
        'Jr. KG': {'day': 'tue', 'period': '2nd'},  # Jr. KG Assembly during 2nd period
        'Sr. KG': {'day': 'tue', 'period': '2nd'},  # Sr. KG Assembly during 2nd period
        'I': {'day': 'tue', 'period': '5th'}       # Class I Assembly during 5th period
    }

    # Iterate over classes and sections to add "Assembly"
    for cls, sections in timetable.items():
        if cls in assembly_schedule:  # Check if the class has an assembly schedule
            assembly_day = assembly_schedule[cls]['day']
            assembly_period = assembly_schedule[cls]['period']
            for section, df in sections.items():
                if assembly_day in df.index and assembly_period in df.columns:
                    df.loc[assembly_day, assembly_period] = 'Assembly'
    return timetable



classes = ['Jr. KG', 'Sr. KG', 'I']
sections = ['A', 'B', 'C', 'D']
timetable = initialize_timetable(classes, sections)

def pair_half_frequency_subjects(period_allotment):
    paired_subjects = {}
    for cls in ['Jr. KG', 'Sr. KG', 'I']:
        # Extract subjects with 0.5 frequency for this class
        half_freq_subjects = period_allotment[['Subject', cls]]
        # Skip rows where frequency is NaN
        subjects = half_freq_subjects[half_freq_subjects[cls] == 0.5]['Subject'].dropna().tolist()
        # Print subjects with 0.5 frequency for debugging
        #print(f"0.5 Frequency Subjects for {cls}: {subjects}")
        # Handle odd-length lists by leaving the last subject unpaired
        paired_subjects[cls] = [
            (subjects[i], subjects[i+1])
            for i in range(0, len(subjects) - 1, 2)
        ]
        # If there is an unpaired subject, append it with itself
        if len(subjects) % 2 == 1:
            paired_subjects[cls].append((subjects[-1], subjects[-1]))
    return paired_subjects
# Generate pairs
paired_subjects = pair_half_frequency_subjects(period_allotment)
print(paired_subjects)


# def allocate_combined_half_frequency_subjects(timetable, paired_subjects):
#     # Iterate over each class
#     for cls, pairs in paired_subjects.items():
#         # Combine each pair into a single unit (e.g., "Computer Studies / Circle Time")
#         combined_pairs = [f"{pair[0]} / {pair[1]}" for pair in pairs]
#         print(f"Processing class: {cls}")
#         print(f"Combined pairs for {cls}: {combined_pairs}")

#         # Assign combined pairs to the timetable
#         for section, days in timetable[cls].items():
#             print(f"Processing {cls} - {section}: {days}")

#             for day, periods in days.items():
#                 # Find empty slots in the timetable
#                 empty_slots = [i for i, period in enumerate(periods) if period.strip() == '']

#                 # Allocate combined pairs to empty slots only once
#                 for pair in combined_pairs:
#                     if empty_slots:
#                         # Assign the pair to the first available empty slot
#                         periods[empty_slots[0]] = pair
#                         print(f"Allocated '{pair}' to {cls} on {day}, slot {empty_slots[0]}")
#                         # Remove the used slot from the list
#                         empty_slots.pop(0)
#                     # If no empty slots are left, stop allocation for the day
#                     if not empty_slots:
#                         break

#     return timetable

def allocate_combined_half_frequency_subjects(timetable, paired_subjects):
    # Iterate over each class
    for cls, pairs in paired_subjects.items():
        # Combine each pair into a single unit (e.g., "Computer Studies / Circle Time")
        combined_pairs = [f"{pair[0]} / {pair[1]}" for pair in pairs]
        print(f"Processing class: {cls}")
        print(f"Combined pairs for {cls}: {combined_pairs}")

        # Assign combined pairs to the timetable
        for section, days in timetable[cls].items():
            print(f"Processing {cls} - {section}: {days}")

            # Create a set to track already allocated pairs for each day
            allocated_pairs = set()

            for day, periods in days.items():
                if day in ['break', 'dispersal']:
                    print(f"Skipping {day} for {cls} as it is not a class day.")
                    continue
                print(f"Processing day: {day}")
                # Find empty slots in the timetable
                empty_slots = [i for i, period in enumerate(periods) if period.strip() == '']
                print(f"Empty slots for {cls} on {day}: {empty_slots}")

                # If no empty slots are found, continue to next day
                if not empty_slots:
                    print(f"No empty slots for {cls} on {day}, skipping.")
                    continue

                # Allocate combined pairs to empty slots only once
                for pair in combined_pairs:
                    if pair not in allocated_pairs and empty_slots:
                        # Assign the pair to the first available empty slot
                        periods[empty_slots[0]] = pair
                        print(f"Allocated '{pair}' to {cls} on {day}, slot {empty_slots[0]}")
                        # Mark this pair as allocated
                        allocated_pairs.add(pair)
                        # Remove the used slot from the list
                        empty_slots.pop(0)

                        # If all pairs are allocated, break out of the loop
                        if len(allocated_pairs) == len(combined_pairs):
                            print(f"All pairs allocated for {cls} on {day}")
                            break
                for i in empty_slots:
                    periods[i] = "leisure"

    return timetable


# Helper function for subject allocation
# def allocate_subjects(timetable, period_allotment):
#     period_allotment = {subject: freq for subject, freq in period_allotment.items() if subject != 'Assembly'}
#     for subject, frequency in period_allotment.items():
#         periods_allocated = 0
#         # Iterate over each period (slot)
#         for slot in timetable.columns:
#             if slot in ['break', 'dispersal', 'HR']:  # Skip non-teaching periods
#                 continue
#             # Iterate over each day for the current period (slot)
#             for day in timetable.index:
#                 if periods_allocated >= frequency:  # Stop if the subject's frequency is met
#                     break
#                 if timetable.at[day, slot].strip() == '':  # If the slot is empty
#                     timetable.at[day, slot] = subject
#                     periods_allocated += 1
#                     if periods_allocated >= frequency:  # Stop if frequency is met
#                         break
#             if periods_allocated >= frequency:  # Stop if the subject's frequency is met
#                 break
#         # Check if all periods were allocated, if not, issue a warning
#         if periods_allocated < frequency:
#             print(f"Warning: Not enough slots to allocate {frequency} periods of {subject}. Allocated {periods_allocated} periods.")
#     return timetable

def allocate_subjects(timetable, period_allotment):
    # Remove 'Assembly' from the allocation
    period_allotment = {subject: freq for subject, freq in period_allotment.items() if subject != 'Assembly' and freq != 0.5}
    for subject, frequency in period_allotment.items():
        periods_allocated = 0
        # Iterate over each period (slot)
        for slot in timetable.columns:
            if slot in ['break', 'dispersal', 'HomeRoom']:  # Skip non-teaching periods
                continue
            # Iterate over each day for the current period (slot)
            for day in timetable.index:
                if periods_allocated >= frequency:  # Stop if the subject's frequency is met
                    break
                # Check for an empty slot
                if timetable.at[day, slot].strip() == '':  # If the slot is empty
                    timetable.at[day, slot] = subject
                    periods_allocated += 1
                if periods_allocated >= frequency:  # Stop if frequency is met
                    break
            if periods_allocated >= frequency:  # Stop if the subject's frequency is met
                break

        # Check if all periods were allocated, if not, issue a warning
        if periods_allocated < frequency:
            print(f"Warning: Not enough slots to allocate {frequency} periods of {subject}. Allocated {periods_allocated} periods.")

    return timetable



def assign_teachers(timetable, teacher_subject_mapping):
    for cls in classes:
        for section in sections:
            # Remove spaces when constructing class_section for "I" classes
            class_section = f"{cls}{' ' if cls != 'I' else ''}{section}"
            if class_section not in teacher_subject_mapping.columns:
                print(f"Class-Section {class_section} not found in teacher_subject_mapping.")
                continue

            for day in rows:
                for col in columns:
                    if col not in ['break', 'dispersal', 'HomeRoom']:
                        subject = timetable[cls][section].loc[day, col]
                        if subject != '':
                            # print("in") # Debug statement Get the teacher(s)
                            # for this subject and class-section
                            teachers = teacher_subject_mapping.loc[
                                teacher_subject_mapping['Subject'] == subject,
                                class_section
                            ].values
                            if teachers.size > 0:
                                timetable[cls][section].loc[day, col] += f" ({teachers[0]})"

add_assembly(timetable)
# Debugging subject allocation
for cls in classes:
    for section in sections:
        if cls in period_allotment.columns:
            # Extract period allotment for the specific class
            period_allotment_class = period_allotment[['Subject', cls]]
            period_allotment_class = period_allotment_class.set_index('Subject').to_dict()[cls]

            # Allocate subjects to the timetable
            timetable[cls][section] = allocate_subjects(timetable[cls][section], period_allotment_class)
        else:
            print(f"Class {cls} not found in period_allotment columns")

# Debugging teacher assignment
timetable = allocate_combined_half_frequency_subjects(timetable, paired_subjects)
assign_teachers(timetable, teacher_subject_mapping)
fill_common_columns(timetable)




def assign_teachers_with_alternate_hr(timetable, teacher_subject_mapping):
    # Create a mock DataFrame for home room teachers# Create a mock DataFrame for home room teachers
    data = {
    'Class-Section': ['Jr. KG A', 'Jr. KG B', 'Jr. KG C', 'Jr. KG D', 'Sr. KG A', 'Sr. KG B', 'Sr. KG C', 'Sr. KG D', 'I A', 'I B', 'I C', 'I D'],
    'Teachers': [['Kim', 'Rukhsaar'], ['Maria', 'Shital'], ['Shireen', 'Uphaar'], ['Priyanka', 'Darshana'],
                 ['Pooja', 'Nupur'], ['Anjum', 'Lakshmipriya'], ['Shubha', 'Jyoti'], ['Ankita', 'Bhakti'],
                 ['Remya', 'Saraswati'], ['Kanak', 'Matilda'], ['Rishita', 'Shashwati'], ['Charu', 'Maariyah']]
    }
    # Convert the "Home room Teachers" data into a dictionary for easy lookup
    home_teachers_df = pd.DataFrame(data)
    home_teachers_mapping = dict(zip(home_teachers_df['Class-Section'], home_teachers_df['Teachers']))

    classes = list(timetable.keys())  # Extract classes from timetable keys
    sections = list(timetable[classes[0]].keys())  # Extract sections from one class's timetable

    for cls in classes:
        for section in sections:
            # Construct class-section key
            class_section = f"{cls} {section}"
            if class_section not in home_teachers_mapping:
                print(f"Class-Section {class_section} not found in home_teachers_mapping.")
                continue

            # Get the two teachers for this class-section
            hr_teachers = home_teachers_mapping[class_section]
            if len(hr_teachers) < 2:
                print(f"Not enough teachers for HR mapping in {class_section}.")
                continue

            # Initialize alternate HR mapping
            alternate_teacher_index = 0
            rows = timetable[cls][section].index
            columns = timetable[cls][section].columns

            for day in rows:
                for col in columns:
                    if col not in ['break', 'dispersal', 'HomeRoom']:
                        subject = timetable[cls][section].loc[day, col]
                        if "HR" in subject:  # Check for HR
                            # Assign alternate HR teacher
                            teacher = hr_teachers[alternate_teacher_index]
                            timetable[cls][section].loc[day, col] = subject.replace("HR", teacher)
                            # Switch to the next teacher for alternate mapping
                            alternate_teacher_index = (alternate_teacher_index + 1) % len(hr_teachers)

    return timetable

assign_teachers_with_alternate_hr(timetable=timetable, teacher_subject_mapping=teacher_subject_mapping)

# Step 9: Save the timetables to Excel
writer = pd.ExcelWriter('/content/Generated_Timetables.xlsx', engine='xlsxwriter')

for cls in classes:
    for section in sections:
        timetable[cls][section].to_excel(writer, sheet_name=f"{cls}_{section}")

writer.close()
print("Timetables generated and saved successfully.")


{'Jr. KG': [('Computer Studies', 'Circle Time'), ('Life Skill', 'Yoga'), ('PLAY PEN ', 'Role play')], 'Sr. KG': [('Computer Studies', 'Circle Time'), ('Life Skill', 'Yoga'), ('DEED', 'Assembly'), ('PLAY PEN ', 'PLAY PEN ')], 'I': []}
Processing class: Jr. KG
Combined pairs for Jr. KG: ['Computer Studies / Circle Time', 'Life Skill / Yoga', 'PLAY PEN  / Role play']
Processing Jr. KG - A:      1st       2nd          3rd             4th             5th break  \
mon  Eng     Hindi  Mathematic              EVS  Clay modelling         
tue  Eng  Assembly  Mathematic   PE/ CCA Sports           Music         
wed  Eng     Hindi  Mathematic   PE/ CCA Sports           Dance         
thu  Eng     Hindi  Mathematic             Art          Library         
fri  Eng     Hindi          EVS            Art             DEED         

                              6th 7th dispersal  
mon                   Wonder time                
tue  Environmental Education --EE                
wed  Environmental Ed

<ipython-input-3-ca075d6d4191>:190: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-3-ca075d6d4191>:190: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-3-ca075d6d4191>:190: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  periods[empty_slots[0]] = pair
<ipython-input-3-ca075d6d4191>:202: FutureWarning: Series.__setitem__ treating key

Processing day: 1st
Empty slots for I on 1st: []
No empty slots for I on 1st, skipping.
Processing day: 2nd
Empty slots for I on 2nd: []
No empty slots for I on 2nd, skipping.
Skipping break for I as it is not a class day.
Processing day: 3rd
Empty slots for I on 3rd: []
No empty slots for I on 3rd, skipping.
Processing day: 4th
Empty slots for I on 4th: []
No empty slots for I on 4th, skipping.
Processing day: 5th
Empty slots for I on 5th: []
No empty slots for I on 5th, skipping.
Processing day: 6th
Empty slots for I on 6th: []
No empty slots for I on 6th, skipping.
Processing day: 7th
Empty slots for I on 7th: []
No empty slots for I on 7th, skipping.
Processing day: 8th
Empty slots for I on 8th: []
No empty slots for I on 8th, skipping.
Skipping break for I as it is not a class day.
Processing day: 9th
Empty slots for I on 9th: []
No empty slots for I on 9th, skipping.
Processing day: 10th
Empty slots for I on 10th: []
No empty slots for I on 10th, skipping.
Skipping dispersal for 